In [1]:
import skimage
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import PIL
#from PIL import ImageFilter, Image 
import math
import time
import scipy as sp
#from scipy.optimize import curve_fit
import PySpin
import cv2

In [444]:
def gauss(x, A, mu, sigma, off):
    return A * np.exp(-(x-mu)**2 / (2 * sigma**2)) + off


In [445]:
ppmm = 1/0.0069  # pixels per mm to convert position to mm
NUM_IMG = 100000
X_Max_Pos = np.zeros(NUM_IMG)   #array to follow the X-position of the beam center
Y_Max_Pos = np.zeros(NUM_IMG)   #array to follow the Y-position of the beam center
pos2 = np.zeros(NUM_IMG)
std2 = np.zeros(NUM_IMG)
FWHM  = np.zeros(NUM_IMG)

In [446]:
#starting the camera
serial = '20270803'  #serisl of the flir cam
system = PySpin.System.GetInstance() # Retrieve singleton reference to system object

version = system.GetLibraryVersion() # Get current library version

cam_list = system.GetCameras()       # Retrieve list of cameras from the system

cam = cam_list.GetBySerial(serial)   
cam.Init()
processor = PySpin.ImageProcessor()
cam.BeginAcquisition()

In [447]:
%%time
%matplotlib qt

fig = plt.figure(1)
ax = None

#fig.canvas.mpl_connect('close_event', handle_close)
for i in range (NUM_IMG):
    image_result = cam.GetNextImage(1000)
    FrameNP =  image_result.GetNDArray() #get the result as a numpy array
    plt.imshow(FrameNP, cmap='gray')
    cam.ExposureTime.GetValue()

    #imax = plt.imshow(FrameNP)
    FramePIL = PIL.Image.fromarray( FrameNP ) #get the result as a PIL image
    image_result.Release() #releases the result to increase speed
    FrameNP = (FrameNP - np.min(FrameNP)) / (np.max(FrameNP) - np.min(FrameNP)) # normalize the matrix
    
    # the elliptic shape of the beam is extracted on the form of points on a certain intensity of the gaussian beam
    yx_coords = np.column_stack(np.where((FrameNP >= 0.49 ) & (FrameNP <= 0.51)))
    
    # the center of this ellipse is in the middle between the max Y and min Y and max X and min X 
    Y_Center = ( (np.amax(yx_coords[:,0]) + np.amin(yx_coords[:,0]) ) / 2)
    X_Center = ( (np.amax(yx_coords[:,1]) + np.amin(yx_coords[:,1]) ) / 2)
    X_Max_Pos[i] = X_Center / ppmm
    Y_Max_Pos[i] = Y_Center / ppmm
    plt.scatter(X_Center,Y_Center, c='b')
    #plt.scatter(yx_coords[:,1],yx_coords[:,0], c='g')
    
    # the axis is set at the furthest point from the center
    d_from_center = (yx_coords[:,1] - X_Center)**2 + (yx_coords[:,0] - Y_Center)**2
    FWHM[i] = 2 * math.sqrt(np.amax(d_from_center)) / ppmm
    P_axis = np.where(d_from_center == np.amax(d_from_center))
    P_axis = yx_coords[P_axis[0]] 

    m = (P_axis[0][0] - Y_Center) / (P_axis[0][1] - X_Center) # the slope of the axis
    
    
    #centering and croping the image
    FramePIL = FramePIL.crop( ( round(X_Center - 190) , round(Y_Center - 190) , round(X_Center + 190)  , round(Y_Center + 190) ) )
    FramePIL = FramePIL.rotate( abs(math.atan(m) * 180 / math.pi) + 90)
    sum_rows = np.mean(FramePIL, axis=0)   #get array with the sums of the rows
    sum_col = np.mean(FramePIL, axis=1)    #get array with the sums of the columns

    x_col=np.linspace(0, FramePIL.height -1 , num = FramePIL.height) / ppmm   # dim in mm
    x_rows=np.linspace(0, FramePIL.width -1 , num = FramePIL.width) / ppmm    # dim in mm

    popt1,pcov1 = sp.optimize.curve_fit(gauss, x_rows,  sum_rows)

    popt2,pcov2 = sp.optimize.curve_fit(gauss, x_col,  sum_col)

    pos2[i] = popt2[1]
    std2[i] = abs(popt2[2])
    plt.pause(0.001)
    plt.clf()
cam.EndAcquisition()
cam.DeInit()

<timed exec>:16: RuntimeWarning: invalid value encountered in divide


ValueError: zero-size array to reduction operation maximum which has no identity

In [442]:
np.mean(X_Max_Pos)

2.2781144535000006

In [448]:
plt.figure
#plt.plot(std2 )
plt.plot( Y_Max_Pos)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [ ]:
cam.EndAcquisition()
cam.DeInit()